In [1]:
import pandas as pd
import csv


In [2]:
traffic_data = "./US_Accidents_March23.csv"

In [3]:
traffic_data

'./US_Accidents_March23.csv'

In [4]:
traffic_df=pd.read_csv(traffic_data)

In [5]:
traffic_df

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.010,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.010,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.010,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.010,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.010,...,False,False,False,False,True,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728389,A-7777757,Source1,2,2019-08-23 18:03:25,2019-08-23 18:32:01,34.002480,-117.379360,33.99888,-117.37094,0.543,...,False,False,False,False,False,False,Day,Day,Day,Day
7728390,A-7777758,Source1,2,2019-08-23 19:11:30,2019-08-23 19:38:23,32.766960,-117.148060,32.76555,-117.15363,0.338,...,False,False,False,False,False,False,Day,Day,Day,Day
7728391,A-7777759,Source1,2,2019-08-23 19:00:21,2019-08-23 19:28:49,33.775450,-117.847790,33.77740,-117.85727,0.561,...,False,False,False,False,False,False,Day,Day,Day,Day
7728392,A-7777760,Source1,2,2019-08-23 19:00:21,2019-08-23 19:29:42,33.992460,-118.403020,33.98311,-118.39565,0.772,...,False,False,False,False,False,False,Day,Day,Day,Day


In [6]:
list(traffic_df)

['ID',
 'Source',
 'Severity',
 'Start_Time',
 'End_Time',
 'Start_Lat',
 'Start_Lng',
 'End_Lat',
 'End_Lng',
 'Distance(mi)',
 'Description',
 'Street',
 'City',
 'County',
 'State',
 'Zipcode',
 'Country',
 'Timezone',
 'Airport_Code',
 'Weather_Timestamp',
 'Temperature(F)',
 'Wind_Chill(F)',
 'Humidity(%)',
 'Pressure(in)',
 'Visibility(mi)',
 'Wind_Direction',
 'Wind_Speed(mph)',
 'Precipitation(in)',
 'Weather_Condition',
 'Amenity',
 'Bump',
 'Crossing',
 'Give_Way',
 'Junction',
 'No_Exit',
 'Railway',
 'Roundabout',
 'Station',
 'Stop',
 'Traffic_Calming',
 'Traffic_Signal',
 'Turning_Loop',
 'Sunrise_Sunset',
 'Civil_Twilight',
 'Nautical_Twilight',
 'Astronomical_Twilight']

In [8]:
reduced_traffic_df = traffic_df[["Start_Time","End_Time","Severity","City","State","Weather_Condition",
                                 "Temperature(F)","Visibility(mi)","Wind_Speed(mph)","Precipitation(in)",\
                                 "Sunrise_Sunset","Description","Bump","Crossing", "Give_Way","Junction",\
                                 "No_Exit","Railway","Roundabout","Station","Stop","Traffic_Calming",\
                                 "Traffic_Signal","Turning_Loop"
                                ]]
reduced_traffic_df

,Start_Time,End_Time,Severity,City,State,Weather_Condition,Temperature(F),Visibility(mi),Wind_Speed(mph),Precipitation(in),...,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop
0,2016-02-08 05:46:00,2016-02-08 11:00:00,3,Dayton,OH,Light Rain,36.9,10.0,NaN,0.02,...,False,False,False,False,False,False,False,False,False,False
1,2016-02-08 06:07:59,2016-02-08 06:37:59,2,Reynoldsburg,OH,Light Rain,37.9,10.0,NaN,0.00,...,False,False,False,False,False,False,False,False,False,False
2,2016-02-08 06:49:27,2016-02-08 07:19:27,2,Williamsburg,OH,Overcast,36.0,10.0,3.5,NaN,...,False,False,False,False,False,False,False,False,True,False
3,2016-02-08 07:23:34,2016-02-08 07:53:34,3,Dayton,OH,Mostly Cloudy,35.1,9.0,4.6,NaN,...,False,False,False,False,False,False,False,False,False,False
4,2016-02-08 07:39:07,2016-02-08 08:09:07,2,Dayton,OH,Mostly Cloudy,36.0,6.0,3.5,NaN,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728389,2019-08-23 18:03:25,2019-08-23 18:32:01,2,Riverside,CA,Fair,86.0,10.0,13.0,0.00,...,False,False,False,False,False,False,False,False,False,False
7728390,2019-08-23 19:11:30,2019-08-23 19:38:23,2,San Diego,CA,Fair,70.0,10.0,6.0,0.00,...,False,False,False,False,False,False,False,False,False,False
7728391,2019-08-23 19:00:21,2019-08-23 19:28:49,2,Orange,CA,Partly Cloudy,73.0,10.0,10.0,0.00,...,False,True,False,False,False,False,False,False,False,False
7728392,2019-08-23 19:00:21,2019-08-23 19:29:42,2,Culver City,CA,Fair,71.0,10.0,8.0,0.00,...,False,False,False,False,False,False,False,False,False,False


In [9]:
reduced_traffic_df["Start_Date"]=pd.to_datetime(reduced_traffic_df["Start_Time"],errors='coerce')
reduced_traffic_df

/var/folders/g7/8_nfr5nn6_d576ng21c0x7x40000gn/T/ipykernel_27787/3821547331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_traffic_df["Start_Date"]=pd.to_datetime(reduced_traffic_df["Start_Time"],errors='coerce')


,Start_Time,End_Time,Severity,City,State,Weather_Condition,Temperature(F),Visibility(mi),Wind_Speed(mph),Precipitation(in),...,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Start_Date
0,2016-02-08 05:46:00,2016-02-08 11:00:00,3,Dayton,OH,Light Rain,36.9,10.0,NaN,0.02,...,False,False,False,False,False,False,False,False,False,2016-02-08 05:46:00
1,2016-02-08 06:07:59,2016-02-08 06:37:59,2,Reynoldsburg,OH,Light Rain,37.9,10.0,NaN,0.00,...,False,False,False,False,False,False,False,False,False,2016-02-08 06:07:59
2,2016-02-08 06:49:27,2016-02-08 07:19:27,2,Williamsburg,OH,Overcast,36.0,10.0,3.5,NaN,...,False,False,False,False,False,False,False,True,False,2016-02-08 06:49:27
3,2016-02-08 07:23:34,2016-02-08 07:53:34,3,Dayton,OH,Mostly Cloudy,35.1,9.0,4.6,NaN,...,False,False,False,False,False,False,False,False,False,2016-02-08 07:23:34
4,2016-02-08 07:39:07,2016-02-08 08:09:07,2,Dayton,OH,Mostly Cloudy,36.0,6.0,3.5,NaN,...,False,False,False,False,False,False,False,True,False,2016-02-08 07:39:07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728389,2019-08-23 18:03:25,2019-08-23 18:32:01,2,Riverside,CA,Fair,86.0,10.0,13.0,0.00,...,False,False,False,False,False,False,False,False,False,2019-08-23 18:03:25
7728390,2019-08-23 19:11:30,2019-08-23 19:38:23,2,San Diego,CA,Fair,70.0,10.0,6.0,0.00,...,False,False,False,False,False,False,False,False,False,2019-08-23 19:11:30
7728391,2019-08-23 19:00:21,2019-08-23 19:28:49,2,Orange,CA,Partly Cloudy,73.0,10.0,10.0,0.00,...,True,False,False,False,False,False,False,False,False,2019-08-23 19:00:21
7728392,2019-08-23 19:00:21,2019-08-23 19:29:42,2,Culver City,CA,Fair,71.0,10.0,8.0,0.00,...,False,False,False,False,False,False,False,False,False,2019-08-23 19:00:21


In [10]:
reduced_traffic_df["Year"]=reduced_traffic_df["Start_Date"].dt.strftime("%Y")
reduced_traffic_df

/var/folders/g7/8_nfr5nn6_d576ng21c0x7x40000gn/T/ipykernel_27787/1415854064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_traffic_df["Year"]=reduced_traffic_df["Start_Date"].dt.strftime("%Y")


,Start_Time,End_Time,Severity,City,State,Weather_Condition,Temperature(F),Visibility(mi),Wind_Speed(mph),Precipitation(in),...,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Start_Date,Year
0,2016-02-08 05:46:00,2016-02-08 11:00:00,3,Dayton,OH,Light Rain,36.9,10.0,NaN,0.02,...,False,False,False,False,False,False,False,False,2016-02-08 05:46:00,2016
1,2016-02-08 06:07:59,2016-02-08 06:37:59,2,Reynoldsburg,OH,Light Rain,37.9,10.0,NaN,0.00,...,False,False,False,False,False,False,False,False,2016-02-08 06:07:59,2016
2,2016-02-08 06:49:27,2016-02-08 07:19:27,2,Williamsburg,OH,Overcast,36.0,10.0,3.5,NaN,...,False,False,False,False,False,False,True,False,2016-02-08 06:49:27,2016
3,2016-02-08 07:23:34,2016-02-08 07:53:34,3,Dayton,OH,Mostly Cloudy,35.1,9.0,4.6,NaN,...,False,False,False,False,False,False,False,False,2016-02-08 07:23:34,2016
4,2016-02-08 07:39:07,2016-02-08 08:09:07,2,Dayton,OH,Mostly Cloudy,36.0,6.0,3.5,NaN,...,False,False,False,False,False,False,True,False,2016-02-08 07:39:07,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728389,2019-08-23 18:03:25,2019-08-23 18:32:01,2,Riverside,CA,Fair,86.0,10.0,13.0,0.00,...,False,False,False,False,False,False,False,False,2019-08-23 18:03:25,2019
7728390,2019-08-23 19:11:30,2019-08-23 19:38:23,2,San Diego,CA,Fair,70.0,10.0,6.0,0.00,...,False,False,False,False,False,False,False,False,2019-08-23 19:11:30,2019
7728391,2019-08-23 19:00:21,2019-08-23 19:28:49,2,Orange,CA,Partly Cloudy,73.0,10.0,10.0,0.00,...,False,False,False,False,False,False,False,False,2019-08-23 19:00:21,2019
7728392,2019-08-23 19:00:21,2019-08-23 19:29:42,2,Culver City,CA,Fair,71.0,10.0,8.0,0.00,...,False,False,False,False,False,False,False,False,2019-08-23 19:00:21,2019


In [21]:
traffic_2022 = reduced_traffic_df[(reduced_traffic_df["Start_Time"] >= '2022-01-01 00:00:00')
                          & (reduced_traffic_df["End_Time"] <= '2022-12-31 23:59:59')]
traffic_2022

,Start_Time,End_Time,Severity,City,State,Weather_Condition,Temperature(F),Visibility(mi),Wind_Speed(mph),Precipitation(in),...,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Start_Date,Year
512217,2022-09-08 05:49:30,2022-09-08 06:34:53,1,Bartlett,IL,Fair,58.0,10.0,0.0,0.0,...,False,False,False,False,False,False,True,False,2022-09-08 05:49:30,2022
512218,2022-09-08 02:02:05,2022-09-08 04:31:32,1,Littlerock,CA,Fair,86.0,10.0,6.0,0.0,...,False,False,False,False,False,False,False,False,2022-09-08 02:02:05,2022
512219,2022-09-08 05:14:12,2022-09-08 07:38:17,1,Richmond,VA,Mostly Cloudy,68.0,10.0,8.0,0.0,...,False,False,False,False,False,False,True,False,2022-09-08 05:14:12,2022
512220,2022-09-08 06:22:57,2022-09-08 06:52:42,1,Alliance,OH,Mostly Cloudy,62.0,7.0,6.0,0.0,...,False,False,False,False,False,False,False,False,2022-09-08 06:22:57,2022
512221,2022-09-08 06:36:20,2022-09-08 07:05:58,2,Independence,OH,Partly Cloudy,63.0,7.0,3.0,0.0,...,False,False,False,False,False,False,False,False,2022-09-08 06:36:20,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5423492,2022-10-08 13:29:50.000000000,2022-10-08 14:44:50.000000000,2,Dallas,TX,Partly Cloudy,75.0,10.0,6.0,0.0,...,False,False,False,False,False,False,True,False,NaT,NaN
5423493,2022-10-11 05:30:00.000000000,2022-10-11 06:47:57.000000000,2,Riverside,CA,Cloudy,64.0,5.0,0.0,0.0,...,False,False,False,False,False,False,False,False,NaT,NaN
5423494,2022-08-25 09:41:00.000000000,2022-08-25 11:00:54.000000000,2,Riverside,CA,Fair,83.0,10.0,6.0,0.0,...,False,False,False,False,False,False,True,False,NaT,NaN
5423495,2022-06-28 14:18:09.000000000,2022-06-28 19:29:33.000000000,2,Merrick,NY,Mostly Cloudy,75.0,10.0,0.0,0.0,...,False,False,False,False,False,False,False,False,NaT,NaN


In [12]:
reduced_traffic_df["Year"].value_counts().sort_values()

Year
2023     166552
2016     410821
2017     717290
2018     893426
2019     954302
2020    1161598
2022    1268806
2021    1412433
Name: count, dtype: int64

In [13]:
traffic_2022["State"].unique()

array(['IL', 'CA', 'VA', 'OH', 'PA', 'SC', 'NJ', 'NY', 'FL', 'NC', 'TX',
       'AZ', 'TN', 'MA', 'WA', 'AL', 'KY', 'GA', 'MI', 'LA', 'IN', 'CT',
       'MN', 'OR', 'CO', 'OK', 'NV', 'MD', 'UT', 'MO', 'KS', 'NM', 'AR',
       'NH', 'NE', 'IA', 'WI', 'RI', 'DE', 'MS', 'ME', 'DC', 'VT', 'WV',
       'WY', 'ID', 'ND', 'MT', 'SD'], dtype=object)

In [ ]:
traffic_2022_NY=traffic_2022[traffic_2022["State"]=='NY']
traffic_2022_NY

In [14]:
reduced_traffic_df["State"].value_counts()

State
CA    1741433
FL     880192
TX     582837
SC     382557
NY     347960
NC     338199
VA     303301
PA     296620
MN     192084
OR     179660
AZ     170609
GA     169234
IL     168958
TN     167388
MI     162191
LA     149701
NJ     140719
MD     140417
OH     118115
WA     108221
AL     101044
UT      97079
CO      90885
OK      83647
MO      77323
CT      71005
IN      67224
MA      61996
WI      34688
KY      32254
NE      28870
MT      28496
IA      26307
AR      22780
NV      21665
KS      20992
DC      18630
RI      16971
MS      15181
DE      14097
WV      13793
ID      11376
NM      10325
NH      10213
WY       3757
ND       3487
ME       2698
VT        926
SD        289
Name: count, dtype: int64

In [15]:
city_traffic = reduced_traffic_df.groupby("City")
city_traffic_count = city_traffic.count().sort_values(by=["State"],ascending=False)
city_traffic_count.head(10)
#[city_traffic_count["State"] >=100000]


,Start_Time,End_Time,Severity,State,Weather_Condition,Temperature(F),Visibility(mi),Wind_Speed(mph),Precipitation(in),Sunrise_Sunset,...,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Start_Date,Year
City,,,,,,,,,,,,,,,,,,,,,
Miami,186917,186917,186917,186917,185332,184685,185108,182739,165196,186916,...,186917,186917,186917,186917,186917,186917,186917,186917,156542,156542
Houston,169609,169609,169609,169609,167261,167316,167383,156082,96840,169604,...,169609,169609,169609,169609,169609,169609,169609,169609,159960,159960
Los Angeles,156491,156491,156491,156491,155841,155386,155807,132502,109020,156491,...,156491,156491,156491,156491,156491,156491,156491,156491,142452,142452
Charlotte,138652,138652,138652,138652,138427,138452,138396,128505,92772,138652,...,138652,138652,138652,138652,138652,138652,138652,138652,130532,130532
Dallas,130939,130939,130939,130939,130467,130648,130623,127056,89137,130939,...,130939,130939,130939,130939,130939,130939,130939,130939,120981,120981
Orlando,109733,109733,109733,109733,109312,109237,109272,106812,89736,109733,...,109733,109733,109733,109733,109733,109733,109733,109733,93618,93618
Austin,97359,97359,97359,97359,95672,96197,95532,86270,55790,97359,...,97359,97359,97359,97359,97359,97359,97359,97359,92024,92024
Raleigh,86079,86079,86079,86079,85961,85951,85959,81074,60229,86079,...,86079,86079,86079,86079,86079,86079,86079,86079,79489,79489
Nashville,72930,72930,72930,72930,72842,72842,72840,70125,55903,72930,...,72930,72930,72930,72930,72930,72930,72930,72930,67107,67107


In [ ]:
reduced_traffic_df[reduced_traffic_df["City"]=="Baton Rouge"]

In [28]:
from api_key import census_api
import json
import requests

In [42]:
base_url = 'https://api.census.gov/data/2021/acs/acs5&for=place:*'

print(f"{base_url}&key={census_api}")
print(base_url)

https://api.census.gov/data/2021/acs/acs5&for=place:*&key=9adc74e4cfc4d4a300ce521d201d867bcab7e31a
https://api.census.gov/data/2021/acs/acs5&for=place:*


In [36]:
base_url = 'https://api.census.gov/data/2021/acs/acs5/sptprofile'
api_url = f"{base_url}&key={census_api}"
json_url = requests.get(api_url).json()

In [45]:
test = "https://api.census.gov/data/2021/acs/acs1?get=NAME,B02015_009E,B02015_009M&for=state:*"
print(test)

https://api.census.gov/data/2021/acs/acs1?get=NAME,B02015_009E,B02015_009M&for=state:*
